## gseapy analysis of CM7_CM1E2d56col_unenr123_rawextract_2017
gsea algorithm: http://software.broadinstitute.org/gsea/doc/GSEAUserGuideFrame.html  
gseapy: https://media.readthedocs.org/pdf/gseapy/latest/gseapy.pdf  
https://github.com/stuppie/CM7_CM1E2d56col_unenr123_rawextract_2017/blob/master/scripts/gseapy.ipynb

In [2]:
import sys
import os
from itertools import chain
from collections import defaultdict
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision', 3)
import gseapy as gp
import goatools

obodag = goatools.obo_parser.GODag('go-basic.obo')

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2017-03-16) 48,478 GO Terms


In [3]:
sys.path.insert(0, "/home/gstupp/projects/metaproteomics")
from metaproteomics import utils
#from metaproteomics.analysis import build_loci

BASE = '../out/'
grouped_loci = utils.load(os.path.join(BASE,"grouped_loci_filt_norm.pkl.gz"))

In [4]:
def make_go2Gene_map(grouped_loci, ontology='MF'):    

    ontology_map = {'MF': 'molecular_function', 'BP': 'biological_process', 'CC': 'cellular_component'}
    
    out = defaultdict(set)    
    for l in grouped_loci:
        if 'go' in l.annotations:
            for go in l.annotations['go']:
                if obodag[go].namespace == ontology_map[ontology]:
                    out[go].add(l.cluster_id)
                    for parent in obodag[go].get_all_parents():
                        if obodag[parent].namespace == ontology_map[ontology]:
                            out[parent].add(l.cluster_id)
                
    return dict(out)

def filter_go2gene_map(go_locus):
    
    # Remove "very broad" gene sets. Arbitrary definition: gene sets that emcompass >50% of all IDs
    all_ids = set(chain(*go_locus.values()))
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) / len(all_ids) <= 0.5}

    # Remove terms with less than 5 members: changed from 10 to 5 becasue small #s of proteins compared
    # to what you would find wiht genes
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) >= 5}

    # Remove child terms with identical gene sets as their parents
    to_remove = set()
    for parent in go_locus.keys():
        # If child term has exact same members as parent, remove
        child_ids = [x.id for x in obodag[parent].children if x.id in go_locus.keys()]
        for child in child_ids:
            if go_locus[child] == go_locus[parent]:
                to_remove.add(child)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    # Remove sibling terms with identical gene sets
    to_remove = set()
    to_keep = set()
    for brother in go_locus.keys():
        to_keep.add(brother) # make sure filtered out siblings don't filter out ones we want to keep
        for parent in obodag[brother].parents:
            siblings = set([y.id for y in parent.children])
            siblings.remove(brother)
            for sibling in siblings:
                if sibling in go_locus.keys() and go_locus[brother] == go_locus[sibling] and not sibling in to_keep:
                    to_remove.add(sibling)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    return go_locus

def gomap_to_csv(go2gene, out_file = 'test.tsv'):

    out = ""
    for term, loci in go2gene.items():
        out += "{}\t".format(term)
        out += "{}\t".format(obodag[term].name)
        out += '\t'.join(list(map(str,loci)))
        out += '\n'
        
    with open(out_file, 'w') as fout:
        fout.write(out)

In [5]:
def run_go_gsea(rank_df, g2g_map, seed, outdir='tmp'):
    """
    A ranked df and go2gene mapping returns the result dataframe for GSEA against all go-Terms
    
    loci must be grouped such that avg_ratio and p-values are correct for 1 phenotype
    see rt_unenr_grouped_loci above for example
    """
    import gseapy as gp
        
    # save the go 2 gene map, since gseapy doesn't seem to be able to use one already in memory
    gomap_to_csv(g2g_map, 'temp.gmt')
    
    res = gp.prerank(rnk=rank_df, gene_sets='temp.gmt', outdir=outdir, min_size = 5, max_size=500, 
                     permutation_n = 10000, graph_num = 20, seed=seed)
      
    def get_go_name(term):
        return obodag[term].name
    
    res['name'] = res.index.map(get_go_name)
    
    return res.sort_values('nes', ascending=False)

def plot_gsea_result(row, rank):
    return gp.plot.gsea_plot(rank, row['name'], row.hit_index, row.nes, row.pval, row.fdr, row.rank_ES, phenoPos='Tcell', phenoNeg='RAG')

In [6]:
mf_map = make_go2Gene_map(grouped_loci)
mf_map_f = filter_go2gene_map(mf_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(mf_map), len(mf_map_f)))

bp_map = make_go2Gene_map(grouped_loci, 'BP')
bp_map_f = filter_go2gene_map(bp_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(bp_map), len(bp_map_f)))

cc_map = make_go2Gene_map(grouped_loci, 'CC')
cc_map_f = filter_go2gene_map(cc_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(cc_map), len(cc_map_f)))

Unfiltered: 596	Filtered: 241
Unfiltered: 751	Filtered: 272
Unfiltered: 103	Filtered: 44


## RT vs Control Human/Mouse

In [20]:
out_dir = "RT_control_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_control_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
#cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_control = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_control = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
#cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
#cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-18 11:44:04,172 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-18 11:44:04,222 0005 gene_sets used for further statistical testing.....
2017-03-18 11:44:04,223 Start to run GSEA...Might take a while..................
2017-03-18 11:44:04,952 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-18 11:44:06,638 Congratulations...GSEAPY run successfully...............
2017-03-18 11:44:06,658 Parsing data files for GSEA.............................
2017-03-18 11:44:06,715 0010 gene_sets used for further statistical testi

In [21]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [22]:
mf_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [23]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0065007,0.576,1.796,0.007,0.042,86,14,"[165648111, 165648112, 61499238, 61499613, 615...",biological regulation
GO:0050789,0.609,1.733,0.009,0.038,80,10,"[165648111, 165648112, 61499238, 61503068, 165...",regulation of biological process


In [24]:
bp_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## RT vs Control Non-Human/Mouse

In [25]:
out_dir = "RT_control_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_control_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [26]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_control = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_control = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-18 11:44:18,761 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-18 11:44:18,945 0054 gene_sets used for further statistical testing.....
2017-03-18 11:44:18,946 Start to run GSEA...Might take a while..................
2017-03-18 11:45:40,317 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-18 11:45:47,064 Congratulations...GSEAPY run successfully...............
2017-03-18 11:45:47,083 Parsing data files for GSEA.............................
2017-03-18 11:45:47,283 0063 gene_sets used for further statistical testi

In [27]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0003723,0.497,2.713,0.000e+00,0.000e+00,367,51,"[7655641, 64669875, 20323736, 33709212, 101851...",RNA binding
GO:0004618,0.644,2.616,0.000e+00,5.094e-05,78,19,"[30875488, 17585926, 8107297, 14006785, 454308...",phosphoglycerate kinase activity
GO:0016774,0.582,2.472,0.000e+00,1.019e-04,111,22,"[30875488, 17585926, 8107297, 14006785, 454308...","phosphotransferase activity, carboxyl group as..."
GO:0001882,0.465,2.212,2.914e-04,2.623e-03,398,31,"[5888442, 80382425, 10185112, 21138590, 692059...",nucleoside binding
GO:0016301,0.443,2.174,2.895e-04,3.199e-03,480,34,"[30875488, 17585926, 8107297, 14006785, 454308...",kinase activity
GO:0019001,0.449,2.064,1.937e-03,7.810e-03,248,28,"[5888442, 80382425, 10185112, 21138590, 692059...",guanyl nucleotide binding
GO:0001883,0.449,2.062,1.185e-03,6.782e-03,248,28,"[5888442, 80382425, 10185112, 21138590, 692059...",purine nucleoside binding
GO:0003735,0.334,1.963,1.743e-03,1.406e-02,360,65,"[7655641, 64669875, 20323736, 33709212, 191078...",structural constituent of ribosome
GO:0022892,0.504,1.875,6.516e-03,2.511e-02,89,15,"[3885199, 1994037, 29978920, 33721585, 3821095...",substrate-specific transporter activity


In [28]:
mf_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004553,-0.480,-1.767,0.018,0.345,75,11,"[11310495, 11310502, 167639021, 61777171, 6337...","hydrolase activity, hydrolyzing O-glycosyl com..."
GO:0016798,-0.480,-1.765,0.019,0.175,76,11,"[11310495, 11310502, 167639021, 61777171, 6337...","hydrolase activity, acting on glycosyl bonds"
GO:0050662,-0.328,-1.734,0.017,0.138,235,27,"[29001612, 27290903, 26904631, 29983272, 20853...",coenzyme binding
GO:0009055,-0.541,-1.733,0.027,0.104,38,8,"[63305114, 63379850, 62942791, 49911241, 63626...",electron carrier activity
GO:0016903,-0.366,-1.589,0.048,0.173,282,16,"[36107728, 6997781, 43414939, 3740901, 1586034...","oxidoreductase activity, acting on the aldehyd..."


In [29]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0009132,0.408,2.117,8.412e-04,0.033,304,41,"[30875488, 17585926, 8107297, 14006785, 454308...",nucleoside diphosphate metabolic process
GO:0046939,0.408,2.113,8.461e-04,0.017,304,41,"[30875488, 17585926, 8107297, 14006785, 454308...",nucleotide phosphorylation
GO:0009135,0.408,2.111,9.901e-04,0.012,302,41,"[30875488, 17585926, 8107297, 14006785, 454308...",purine nucleoside diphosphate metabolic process
GO:0072524,0.401,2.091,7.072e-04,0.010,305,42,"[30875488, 17585926, 8107297, 14006785, 454308...",pyridine-containing compound metabolic process
GO:0006733,0.401,2.089,7.050e-04,0.008,305,42,"[30875488, 17585926, 8107297, 14006785, 454308...",oxidoreduction coenzyme metabolic process
GO:0044267,0.352,2.088,0.000e+00,0.007,413,69,"[7655641, 64669875, 20323736, 33709212, 272121...",cellular protein metabolic process
GO:0009141,0.392,2.087,5.639e-04,0.006,381,45,"[30875488, 17585926, 8107297, 14006785, 454308...",nucleoside triphosphate metabolic process
GO:0046034,0.392,2.086,1.533e-03,0.005,379,45,"[30875488, 17585926, 8107297, 14006785, 454308...",ATP metabolic process
GO:0009126,0.385,2.057,1.520e-03,0.007,409,46,"[30875488, 17585926, 8107297, 14006785, 454308...",purine nucleoside monophosphate metabolic process


In [30]:
bp_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [31]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0015934,0.660,2.114,6.653e-04,0.012,63,10,"[7655641, 64669875, 20323736, 33709212, 828372...",large ribosomal subunit
GO:0044444,0.335,2.014,1.710e-03,0.015,422,70,"[7655641, 64669875, 20323736, 33709212, 296297...",cytoplasmic part
GO:0044446,0.491,1.988,2.516e-03,0.013,169,19,"[7655641, 64669875, 20323736, 33709212, 828372...",intracellular organelle part
GO:0044391,0.491,1.987,1.997e-03,0.010,159,19,"[7655641, 64669875, 20323736, 33709212, 828372...",ribosomal subunit
GO:1990904,0.334,1.952,2.281e-03,0.010,360,65,"[7655641, 64669875, 20323736, 33709212, 191078...",ribonucleoprotein complex
GO:0005622,0.377,1.932,3.536e-03,0.010,306,41,"[7655641, 64669875, 20323736, 33709212, 191078...",intracellular
GO:0043232,0.312,1.764,1.199e-02,0.031,320,57,"[7655641, 64669875, 20323736, 33709212, 191078...",intracellular non-membrane-bounded organelle
GO:0005840,0.312,1.762,1.024e-02,0.028,319,57,"[7655641, 64669875, 20323736, 33709212, 191078...",ribosome
GO:0043229,0.312,1.758,9.684e-03,0.025,327,57,"[7655641, 64669875, 20323736, 33709212, 191078...",intracellular organelle


In [32]:
cc_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016020,-0.466,-1.946,0.007,0.043,75,15,"[21357752, 21687303, 167156307, 61699598, 1666...",membrane
GO:0043190,-0.527,-1.786,0.021,0.061,115,9,"[63132923, 47513944, 11992086, 56076521, 17618...",ATP-binding cassette (ABC) transporter complex
GO:0098797,-0.527,-1.770,0.018,0.045,117,9,"[63132923, 47513944, 11992086, 56076521, 17618...",plasma membrane protein complex
GO:0044459,-0.527,-1.770,0.024,0.034,119,9,"[63132923, 47513944, 11992086, 56076521, 17618...",plasma membrane part
GO:1904949,-0.527,-1.767,0.019,0.027,117,9,"[63132923, 47513944, 11992086, 56076521, 17618...",ATPase complex
GO:1990351,-0.527,-1.766,0.021,0.023,117,9,"[63132923, 47513944, 11992086, 56076521, 17618...",transporter complex


## Rag vs WT Human/Mouse

In [33]:
out_dir = "Rag_WT_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"Rag_WT_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
#cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rag = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rag = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
#cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
#cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-18 11:47:59,842 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-18 11:47:59,879 0004 gene_sets used for further statistical testing.....
2017-03-18 11:47:59,880 Start to run GSEA...Might take a while..................
2017-03-18 11:48:00,298 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-18 11:48:01,610 Congratulations...GSEAPY run successfully...............
2017-03-18 11:48:01,630 Parsing data files for GSEA.............................
2017-03-18 11:48:01,670 0004 gene_sets used for further statistical testi

In [34]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0046872,0.531,1.679,0.028,0.087,488,9,"[61501076, 61502914, 61500363, 61501312, 61499...",metal ion binding


In [35]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0005515,-0.864,-1.869,1.363e-04,6.931e-04,108,6,"[165662997, 61499944, 165642270, 165664161, 16...",protein binding


In [36]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [37]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## Rag vs WT Non-Human/Mouse

In [38]:
out_dir = "Rag_WT_gsea"
df = pd.read_csv(os.path.join(BASE,"Rag_WT_results_named_annot.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [39]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rag = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rag = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rag = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_wt = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-18 11:48:21,871 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-18 11:48:22,364 0074 gene_sets used for further statistical testing.....
2017-03-18 11:48:22,365 Start to run GSEA...Might take a while..................
2017-03-18 11:51:20,080 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-18 11:51:27,403 Congratulations...GSEAPY run successfully...............
2017-03-18 11:51:27,422 Parsing data files for GSEA.............................
2017-03-18 11:51:27,925 0104 gene_sets used for further statistical testi

In [40]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016903,0.427,2.458,0.000e+00,1.541e-04,282,75,"[57377692, 57602627, 20987406, 10318713, 61599...","oxidoreductase activity, acting on the aldehyd..."
GO:0009055,0.730,2.457,0.000e+00,7.706e-05,38,12,"[63098496, 64646230, 21834593, 6263025, 630796...",electron carrier activity
GO:0051540,0.479,2.329,0.000e+00,4.110e-04,149,38,"[62751142, 63901915, 167744672, 62114668, 2842...",metal cluster binding
GO:0016820,0.460,2.183,1.898e-04,2.138e-03,190,35,"[62344870, 168161151, 28240379, 40674602, 6254...","hydrolase activity, acting on acid anhydrides,..."
GO:0016868,0.613,2.061,1.738e-03,5.379e-03,79,12,"[167401290, 39279903, 62071220, 168126288, 415...","intramolecular transferase activity, phosphotr..."
GO:0016866,0.613,2.041,2.309e-03,5.381e-03,112,12,"[167401290, 39279903, 62071220, 168126288, 415...",intramolecular transferase activity
GO:0022804,0.426,2.033,1.715e-03,5.075e-03,205,36,"[62344870, 168161151, 28240379, 40674602, 6254...",active transmembrane transporter activity
GO:0016620,0.433,1.951,1.539e-03,9.536e-03,101,30,"[10318713, 6997781, 166781730, 44171740, 61717...","oxidoreductase activity, acting on the aldehyd..."
GO:0022857,0.384,1.849,4.042e-03,1.968e-02,212,37,"[62344870, 168161151, 28240379, 40674602, 6254...",transmembrane transporter activity


In [41]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0097747,-0.653,-4.453,0.000e+00,0.000e+00,345,143,"[61571198, 61950469, 47461425, 15059448, 82917...",RNA polymerase activity
GO:0016779,-0.614,-4.359,0.000e+00,0.000e+00,463,171,"[18523127, 28436439, 40551040, 59156012, 63039...",nucleotidyltransferase activity
GO:0003677,-0.622,-4.303,0.000e+00,0.000e+00,369,155,"[66926567, 68934118, 51907206, 56510025, 68645...",DNA binding
GO:0004634,-0.870,-3.487,0.000e+00,0.000e+00,43,19,"[62743325, 62658191, 22213675, 18120033, 66797...",phosphopyruvate hydratase activity
GO:0000287,-0.669,-3.414,0.000e+00,0.000e+00,113,42,"[39279903, 62071220, 168126288, 41534199, 8102...",magnesium ion binding
GO:0016835,-0.770,-3.278,0.000e+00,0.000e+00,85,23,"[62217311, 57701171, 62743325, 62658191, 20611...",carbon-oxygen lyase activity
GO:0016781,-0.431,-3.183,0.000e+00,0.000e+00,302,218,"[68757582, 48551975, 59573933, 36437635, 64455...","phosphotransferase activity, paired acceptors"
GO:0016301,-0.385,-2.821,0.000e+00,0.000e+00,480,214,"[21799656, 22349603, 62247377, 64925467, 20722...",kinase activity
GO:0016829,-0.486,-2.564,0.000e+00,1.394e-05,279,46,"[15754209, 18137146, 168075980, 62217311, 3701...",lyase activity


In [42]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0040011,0.543,3.214,0.000,0.000,274,85,"[63837786, 165990924, 167953213, 168171488, 16...",locomotion
GO:0048870,0.534,3.151,0.000,0.000,269,81,"[167953213, 168171488, 167966912, 13586931, 62...",cell motility
GO:0006928,0.534,3.138,0.000,0.000,270,81,"[167953213, 168171488, 167966912, 13586931, 62...",movement of cell or subcellular component
GO:0042558,0.648,2.090,0.001,0.007,40,11,"[31379006, 61964112, 62018486, 65135677, 63995...",pteridine-containing compound metabolic process
GO:0006760,0.640,2.016,0.002,0.012,29,10,"[61964112, 62018486, 65135677, 63995561, 17571...",folic acid-containing compound metabolic process
GO:0042398,0.640,2.002,0.002,0.012,29,10,"[61964112, 62018486, 65135677, 63995561, 17571...",cellular modified amino acid biosynthetic process
GO:0006575,0.640,1.988,0.002,0.011,30,10,"[61964112, 62018486, 65135677, 63995561, 17571...",cellular modified amino acid metabolic process
GO:1901605,0.438,1.800,0.010,0.047,110,22,"[63889213, 165956827, 63092817, 166239197, 276...",alpha-amino acid metabolic process
GO:0051179,0.328,1.793,0.005,0.044,332,61,"[167285315, 13587543, 167348034, 165992613, 16...",localization


In [43]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0032774,-0.653,-4.459,0.000e+00,0.000e+00,345,143,"[61571198, 61950469, 47461425, 15059448, 82917...",RNA biosynthetic process
GO:0072350,-0.796,-3.227,0.000e+00,0.000e+00,29,20,"[38181119, 62737033, 40713606, 38860663, 61303...",tricarboxylic acid metabolic process
GO:0009259,-0.498,-2.863,0.000e+00,0.000e+00,414,66,"[167029093, 167526299, 165866251, 62344870, 64...",ribonucleotide metabolic process
GO:0009123,-0.498,-2.862,0.000e+00,0.000e+00,412,66,"[167029093, 167526299, 165866251, 62344870, 64...",nucleoside monophosphate metabolic process
GO:0019693,-0.486,-2.816,0.000e+00,0.000e+00,416,67,"[167029093, 167526299, 63863560, 165866251, 62...",ribose phosphate metabolic process
GO:0009135,-0.498,-2.792,0.000e+00,0.000e+00,302,58,"[167029093, 167526299, 165866251, 64711323, 63...",purine nucleoside diphosphate metabolic process
GO:0046939,-0.498,-2.785,0.000e+00,0.000e+00,304,58,"[167029093, 167526299, 165866251, 64711323, 63...",nucleotide phosphorylation
GO:0009132,-0.498,-2.769,0.000e+00,0.000e+00,304,58,"[167029093, 167526299, 165866251, 64711323, 63...",nucleoside diphosphate metabolic process
GO:0046034,-0.478,-2.750,0.000e+00,0.000e+00,379,64,"[167029093, 167526299, 165866251, 62344870, 64...",ATP metabolic process


In [44]:
cc_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0042995,0.534,3.138,0.000e+00,0.000e+00,269,81,"[167953213, 168171488, 167966912, 13586931, 62...",cell projection
GO:0044463,0.529,2.985,0.000e+00,0.000e+00,229,66,"[167953213, 168171488, 167966912, 13586931, 62...",cell projection part
GO:0044422,0.492,2.964,0.000e+00,0.000e+00,402,90,"[167953213, 168171488, 167966912, 13586931, 62...",organelle part
GO:0044459,0.531,2.390,0.000e+00,2.728e-05,119,29,"[168161151, 28240379, 40674602, 62545344, 2125...",plasma membrane part
GO:1990351,0.531,2.387,0.000e+00,2.182e-05,117,29,"[168161151, 28240379, 40674602, 62545344, 2125...",transporter complex
GO:1904949,0.531,2.386,0.000e+00,1.819e-05,117,29,"[168161151, 28240379, 40674602, 62545344, 2125...",ATPase complex
GO:0098797,0.531,2.386,0.000e+00,1.559e-05,117,29,"[168161151, 28240379, 40674602, 62545344, 2125...",plasma membrane protein complex
GO:0043190,0.531,2.381,0.000e+00,1.364e-05,115,29,"[168161151, 28240379, 40674602, 62545344, 2125...",ATP-binding cassette (ABC) transporter complex
GO:0098796,0.460,2.183,0.000e+00,3.395e-04,171,35,"[62344870, 168161151, 28240379, 40674602, 6254...",membrane protein complex


In [45]:
cc_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0000015,-0.870,-3.496,0.000e+00,0.000e+00,43,19,"[62743325, 62658191, 22213675, 18120033, 66797...",phosphopyruvate hydratase complex
GO:0044445,-0.870,-3.453,0.000e+00,0.000e+00,47,19,"[62743325, 62658191, 22213675, 18120033, 66797...",cytosolic part
GO:1902494,-0.490,-2.699,0.000e+00,0.000e+00,202,56,"[7522635, 22519241, 63652778, 168124324, 16816...",catalytic complex
GO:0044444,-0.430,-2.607,0.000e+00,0.000e+00,422,80,"[165850968, 11332141, 62054342, 67774188, 4009...",cytoplasmic part
GO:0005737,-0.471,-2.194,4.234e-04,4.848e-04,264,31,"[63092817, 166239197, 61703706, 62359585, 6270...",cytoplasm
GO:0043229,-0.340,-1.842,6.066e-03,9.561e-03,327,51,"[165850968, 11332141, 62054342, 67774188, 4009...",intracellular organelle
GO:0043232,-0.340,-1.838,3.035e-03,8.311e-03,320,51,"[165850968, 11332141, 62054342, 67774188, 4009...",intracellular non-membrane-bounded organelle
GO:0005840,-0.328,-1.759,7.912e-03,1.312e-02,319,50,"[165850968, 11332141, 62054342, 67774188, 4009...",ribosome
GO:1990904,-0.291,-1.596,1.878e-02,3.439e-02,360,55,"[165850968, 11332141, 62054342, 67774188, 4009...",ribonucleoprotein complex


## RT vs WT Human/Mouse

In [9]:
out_dir = "RT_WT_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_WT_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-19 18:33:58,528 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-19 18:33:58,589 0007 gene_sets used for further statistical testing.....
2017-03-19 18:33:58,590 Start to run GSEA...Might take a while..................
2017-03-19 18:33:59,703 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-19 18:34:02,091 Congratulations...GSEAPY run successfully...............
2017-03-19 18:34:02,109 Parsing data files for GSEA.............................
2017-03-19 18:34:02,176 0010 gene_sets used for further statistical testi

In [10]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0030246,0.577,1.538,0.044,0.193,31,7,"[165643482, 165647385, 61499891, 165653125, 16...",carbohydrate binding


In [11]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [12]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0065007,0.649,1.997,8.066e-04,0.012,86,11,"[165648111, 165648112, 61499238, 61499613, 615...",biological regulation
GO:0050789,0.663,1.845,6.288e-03,0.021,80,8,"[165648111, 165648112, 61499238, 61503068, 165...",regulation of biological process


In [13]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## RT vs WT Non-Human/Mouse

In [14]:
out_dir = "RT_WT_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_WT_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [15]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_wt = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_wt = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_wt = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-19 18:35:44,605 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-19 18:35:44,949 0076 gene_sets used for further statistical testing.....
2017-03-19 18:35:44,950 Start to run GSEA...Might take a while..................
2017-03-19 18:38:03,046 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-19 18:38:09,803 Congratulations...GSEAPY run successfully...............
2017-03-19 18:38:09,822 Parsing data files for GSEA.............................
2017-03-19 18:38:10,178 0093 gene_sets used for further statistical testi

In [16]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004618,0.625,2.605,0.000e+00,1.942e-04,78,21,"[30875488, 17585926, 8107297, 14006785, 454308...",phosphoglycerate kinase activity
GO:0016774,0.570,2.481,0.000e+00,1.457e-04,111,24,"[30875488, 17585926, 8107297, 14006785, 454308...","phosphotransferase activity, carboxyl group as..."
GO:0003723,0.339,2.072,1.527e-04,1.489e-02,367,79,"[7655641, 64669875, 20323736, 33709212, 573005...",RNA binding
GO:0022892,0.545,2.058,1.214e-03,1.275e-02,89,16,"[3885199, 1994037, 29978920, 33721585, 3821095...",substrate-specific transporter activity
GO:0022857,0.476,1.959,4.743e-03,2.366e-02,212,20,"[3885199, 1994037, 29978920, 33721585, 3821095...",transmembrane transporter activity
GO:0001882,0.356,1.873,4.115e-03,3.969e-02,398,44,"[5888442, 80382425, 5730052, 10185112, 8974409...",nucleoside binding
GO:0016301,0.327,1.845,2.740e-03,4.271e-02,480,57,"[30875488, 17585926, 8107297, 14006785, 454308...",kinase activity
GO:0016614,0.366,1.781,1.184e-02,5.954e-02,150,34,"[10318713, 13684061, 6155443, 7011122, 1114790...","oxidoreductase activity, acting on CH-OH group..."
GO:0003735,0.272,1.748,3.471e-03,6.653e-02,360,95,"[7655641, 64669875, 20323736, 33709212, 191078...",structural constituent of ribosome


In [17]:
mf_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004634,-0.577,-2.335,0.000e+00,0.005,43,16,"[29629717, 23089117, 17564385, 17549174, 20853...",phosphopyruvate hydratase activity
GO:0016798,-0.427,-2.163,2.532e-04,0.014,76,30,"[11310495, 11409356, 13575738, 11310502, 16756...","hydrolase activity, acting on glycosyl bonds"
GO:0016835,-0.468,-2.115,1.449e-03,0.014,85,21,"[29629717, 23089117, 80889105, 17564385, 62217...",carbon-oxygen lyase activity
GO:0030246,-0.575,-2.084,1.827e-03,0.013,31,12,"[18135548, 11310502, 61777171, 20895349, 63655...",carbohydrate binding
GO:0004553,-0.409,-2.055,1.979e-03,0.013,75,29,"[11310495, 11409356, 13575738, 11310502, 16756...","hydrolase activity, hydrolyzing O-glycosyl com..."
GO:0000287,-0.279,-1.534,4.113e-02,0.303,113,39,"[29629717, 9654148, 23089117, 25933508, 175893...",magnesium ion binding
GO:0046872,-0.214,-1.499,3.099e-02,0.307,488,87,"[4715355, 29629717, 10318713, 9654148, 2308911...",metal ion binding


In [18]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0040011,0.362,2.061,6.275e-04,0.058,274,57,"[168171488, 167307068, 13580518, 13586931, 167...",locomotion
GO:0048870,0.362,2.046,0.000e+00,0.035,269,57,"[168171488, 167307068, 13580518, 13586931, 167...",cell motility
GO:0006928,0.362,2.043,9.539e-04,0.024,270,57,"[168171488, 167307068, 13580518, 13586931, 167...",movement of cell or subcellular component
GO:0072350,0.599,1.836,1.027e-02,0.100,29,9,"[13684061, 78477008, 80889105, 26904631, 29983...",tricarboxylic acid metabolic process
GO:0044267,0.280,1.817,3.021e-03,0.093,413,101,"[7655641, 64669875, 20323736, 33709212, 272121...",cellular protein metabolic process
GO:1902578,0.457,1.815,8.095e-03,0.078,132,18,"[3885199, 1994037, 29978920, 33721585, 3821095...",single-organism localization
GO:0006412,0.269,1.737,6.115e-03,0.118,372,96,"[7655641, 64669875, 20323736, 33709212, 191078...",translation
GO:0043043,0.269,1.733,6.947e-03,0.106,377,96,"[7655641, 64669875, 20323736, 33709212, 191078...",peptide biosynthetic process
GO:0006518,0.269,1.732,5.055e-03,0.095,378,96,"[7655641, 64669875, 20323736, 33709212, 191078...",peptide metabolic process


In [19]:
bp_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0009081,-0.577,-1.936,0.006,0.133,47,10,"[20839749, 51213068, 63289385, 51313098, 13442...",branched-chain amino acid metabolic process
GO:0009082,-0.571,-1.659,0.035,0.364,41,7,"[20839749, 51213068, 51313098, 63235820, 63662...",branched-chain amino acid biosynthetic process
GO:0050794,-0.610,-1.657,0.032,0.246,36,6,"[165873031, 167750483, 59827295, 66251821, 377...",regulation of cellular process
GO:0019222,-0.612,-1.644,0.039,0.199,59,6,"[61892960, 10631876, 59827295, 66251821, 37727...",regulation of metabolic process


In [20]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0044446,0.493,2.191,6.741e-04,0.005,169,25,"[7655641, 64669875, 20323736, 33709212, 170351...",intracellular organelle part
GO:0015934,0.642,2.188,7.048e-04,0.003,63,12,"[7655641, 64669875, 20323736, 33709212, 170351...",large ribosomal subunit
GO:0044391,0.493,2.172,6.767e-04,0.002,159,25,"[7655641, 64669875, 20323736, 33709212, 170351...",ribosomal subunit
GO:0042995,0.362,2.062,1.571e-04,0.004,269,57,"[168171488, 167307068, 13580518, 13586931, 167...",cell projection
GO:0044422,0.345,2.038,7.831e-04,0.004,402,67,"[7655641, 64669875, 20323736, 33709212, 170351...",organelle part
GO:0044463,0.350,1.816,6.970e-03,0.019,229,42,"[168171488, 167307068, 13580518, 13586931, 167...",cell projection part
GO:0005622,0.312,1.811,4.740e-03,0.018,306,63,"[7655641, 64669875, 20323736, 33709212, 573005...",intracellular
GO:1990904,0.272,1.742,4.833e-03,0.025,360,95,"[7655641, 64669875, 20323736, 33709212, 191078...",ribonucleoprotein complex
GO:0098796,0.523,1.598,4.273e-02,0.058,171,9,"[1994037, 32363526, 1809293, 268176, 12867643,...",membrane protein complex


In [21]:
cc_wt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0000015,-0.577,-2.341,0.000e+00,5.354e-04,43,16,"[29629717, 23089117, 17564385, 17549174, 20853...",phosphopyruvate hydratase complex
GO:0044445,-0.565,-2.333,2.360e-04,3.407e-04,47,17,"[29629717, 23089117, 17564385, 63672434, 17549...",cytosolic part
GO:1902494,-0.468,-2.254,2.496e-04,4.542e-04,202,26,"[29629717, 23089117, 11310502, 22519241, 43045...",catalytic complex
GO:0016020,-0.391,-1.604,4.121e-02,3.126e-02,75,17,"[21357752, 21687303, 62114668, 64265900, 46871...",membrane


## RT vs Rag Human/Mouse

In [22]:
out_dir = "RT_Rag_hm_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_Rag_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_rag = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_rag = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-19 18:45:41,297 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-19 18:45:41,355 0008 gene_sets used for further statistical testing.....
2017-03-19 18:45:41,355 Start to run GSEA...Might take a while..................
2017-03-19 18:45:44,212 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-19 18:45:47,040 Congratulations...GSEAPY run successfully...............
2017-03-19 18:45:47,059 Parsing data files for GSEA.............................
2017-03-19 18:45:47,123 0013 gene_sets used for further statistical testi

In [23]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0008270,0.654,1.551,0.034,0.257,89,6,"[165648111, 165648112, 61533297, 165637224, 61...",zinc ion binding


In [24]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


In [25]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0065007,0.612,1.788,0.003,0.051,86,13,"[165648111, 165648112, 61499238, 61499613, 615...",biological regulation
GO:0050789,0.653,1.741,0.007,0.042,80,9,"[165648111, 165648112, 61499238, 61503068, 165...",regulation of biological process


In [26]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,


## RT vs Rag Non-Human/Mouse

In [27]:
out_dir = "RT_Rag_gsea"
df = pd.read_csv(os.path.join(BASE,"RT_Rag_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df = df[~df.human_mouse]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [28]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_rag = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_rag = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_rag = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2017-03-19 18:45:58,304 Parsing data files for GSEA.............................
/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2017-03-19 18:45:58,700 0077 gene_sets used for further statistical testing.....
2017-03-19 18:45:58,701 Start to run GSEA...Might take a while..................
2017-03-19 18:49:04,688 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2017-03-19 18:49:12,020 Congratulations...GSEAPY run successfully...............
2017-03-19 18:49:12,039 Parsing data files for GSEA.............................
2017-03-19 18:49:12,430 0090 gene_sets used for further statistical testi

In [29]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016781,0.530,3.223,0.000e+00,0.000e+00,302,195,"[61708790, 61884808, 36565362, 61724660, 51077...","phosphotransferase activity, paired acceptors"
GO:0003723,0.663,3.209,0.000e+00,0.000e+00,367,55,"[7655641, 64669875, 20323736, 33709212, 514122...",RNA binding
GO:0097747,0.621,3.010,0.000e+00,0.000e+00,345,55,"[12999155, 23375966, 6243639, 16965124, 180684...",RNA polymerase activity
GO:0003677,0.583,2.844,0.000e+00,0.000e+00,369,58,"[12999155, 23375966, 6243639, 16965124, 180684...",DNA binding
GO:0016779,0.559,2.839,0.000e+00,0.000e+00,463,67,"[12999155, 23375966, 6243639, 18275642, 618929...",nucleotidyltransferase activity
GO:0016301,0.433,2.667,0.000e+00,0.000e+00,480,207,"[30875488, 17585926, 8107297, 14006785, 454308...",kinase activity
GO:0004618,0.727,2.640,0.000e+00,0.000e+00,78,19,"[30875488, 17585926, 8107297, 14006785, 454308...",phosphoglycerate kinase activity
GO:0001882,0.578,2.562,0.000e+00,0.000e+00,398,38,"[5888442, 80382425, 21138590, 69205977, 101851...",nucleoside binding
GO:0016774,0.626,2.414,0.000e+00,1.213e-05,111,23,"[30875488, 17585926, 8107297, 14006785, 454308...","phosphotransferase activity, carboxyl group as..."


In [30]:
mf_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0009055,-0.646,-2.467,0.000,4.325e-04,38,18,"[63507869, 63623628, 64864375, 63379850, 64572...",electron carrier activity
GO:0051540,-0.597,-2.403,0.000,3.892e-04,149,21,"[80889105, 61600538, 61963309, 61598826, 64109...",metal cluster binding
GO:0016887,-0.411,-2.327,0.000,5.190e-04,249,73,"[1994037, 32363526, 1809293, 268176, 13568664,...",ATPase activity
GO:0016820,-0.366,-1.940,0.001,2.104e-02,190,56,"[1994037, 32363526, 1809293, 268176, 13568664,...","hydrolase activity, acting on acid anhydrides,..."
GO:1901681,-0.574,-1.801,0.012,4.704e-02,80,10,"[3740901, 61600538, 61598826, 61600015, 616019...",sulfur compound binding
GO:0030976,-0.574,-1.790,0.015,4.238e-02,79,10,"[3740901, 61600538, 61598826, 61600015, 616019...",thiamine pyrophosphate binding
GO:0005215,-0.305,-1.790,0.001,3.641e-02,256,83,"[3885199, 1994037, 29978920, 33721585, 3821095...",transporter activity
GO:0022804,-0.321,-1.755,0.006,4.083e-02,205,61,"[1994037, 33721585, 21357752, 32363526, 180929...",active transmembrane transporter activity
GO:0016757,-0.521,-1.750,0.016,3.743e-02,79,12,"[13750734, 61748514, 167871404, 61739923, 5571...","transferase activity, transferring glycosyl gr..."


In [31]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0032774,0.621,2.987,0.000e+00,0.000e+00,345,55,"[12999155, 23375966, 6243639, 16965124, 180684...",RNA biosynthetic process
GO:0046034,0.509,2.566,0.000e+00,0.000e+00,379,66,"[30875488, 17585926, 8107297, 14006785, 454308...",ATP metabolic process
GO:0009141,0.509,2.565,0.000e+00,0.000e+00,381,66,"[30875488, 17585926, 8107297, 14006785, 454308...",nucleoside triphosphate metabolic process
GO:0009123,0.496,2.520,0.000e+00,0.000e+00,412,67,"[30875488, 17585926, 8107297, 14006785, 454308...",nucleoside monophosphate metabolic process
GO:0009126,0.496,2.502,0.000e+00,2.612e-05,409,67,"[30875488, 17585926, 8107297, 14006785, 454308...",purine nucleoside monophosphate metabolic process
GO:0009132,0.504,2.469,0.000e+00,4.354e-05,304,58,"[30875488, 17585926, 8107297, 14006785, 454308...",nucleoside diphosphate metabolic process
GO:0046939,0.504,2.463,0.000e+00,3.732e-05,304,58,"[30875488, 17585926, 8107297, 14006785, 454308...",nucleotide phosphorylation
GO:0044267,0.451,2.461,0.000e+00,3.265e-05,413,99,"[7655641, 64669875, 20323736, 33709212, 664535...",cellular protein metabolic process
GO:0009135,0.504,2.458,0.000e+00,2.902e-05,302,58,"[30875488, 17585926, 8107297, 14006785, 454308...",purine nucleoside diphosphate metabolic process


In [32]:
bp_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0006790,-0.670,-1.993,0.003,0.033,92,9,"[63603065, 45843396, 61901179, 63849179, 65619...",sulfur compound metabolic process
GO:0022900,-0.617,-1.835,0.010,0.066,72,9,"[61600538, 62232545, 61598826, 61600015, 61601...",electron transport chain
GO:0051234,-0.298,-1.762,0.003,0.075,331,88,"[3885199, 1994037, 29978920, 33721585, 3821095...",establishment of localization
GO:0051179,-0.298,-1.759,0.001,0.058,332,88,"[3885199, 1994037, 29978920, 33721585, 3821095...",localization
GO:0040011,-0.398,-1.679,0.020,0.077,274,24,"[167307068, 13580518, 21419243, 48844574, 1681...",locomotion


In [33]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0044444,0.443,2.446,0.000e+00,0.000e+00,422,106,"[7655641, 64669875, 20323736, 33709212, 664535...",cytoplasmic part
GO:1990904,0.446,2.397,0.000e+00,0.000e+00,360,91,"[7655641, 64669875, 20323736, 33709212, 664535...",ribonucleoprotein complex
GO:0005840,0.427,2.263,0.000e+00,1.113e-04,319,82,"[7655641, 64669875, 20323736, 33709212, 664535...",ribosome
GO:0043229,0.427,2.256,0.000e+00,1.043e-04,327,82,"[7655641, 64669875, 20323736, 33709212, 664535...",intracellular organelle
GO:0043232,0.427,2.250,0.000e+00,1.002e-04,320,82,"[7655641, 64669875, 20323736, 33709212, 664535...",intracellular non-membrane-bounded organelle
GO:0044445,0.645,2.130,7.000e-04,3.895e-04,47,14,"[29629717, 23089117, 66137094, 17549174, 23090...",cytosolic part
GO:0000015,0.645,2.126,5.268e-04,3.458e-04,43,14,"[29629717, 23089117, 66137094, 17549174, 23090...",phosphopyruvate hydratase complex
GO:0015934,0.691,2.105,5.253e-04,3.651e-04,63,11,"[7655641, 64669875, 20323736, 33709212, 828372...",large ribosomal subunit
GO:0005737,0.412,1.901,1.276e-03,2.764e-03,264,47,"[4715355, 1488186, 22444626, 80888632, 3372158...",cytoplasm


In [34]:
cc_rag

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0043190,-0.570,-2.912,0.000e+00,0.000,115,47,"[63078357, 6924770, 63452797, 77351967, 106890...",ATP-binding cassette (ABC) transporter complex
GO:1990351,-0.570,-2.910,0.000e+00,0.000,117,47,"[63078357, 6924770, 63452797, 77351967, 106890...",transporter complex
GO:0098797,-0.570,-2.900,0.000e+00,0.000,117,47,"[63078357, 6924770, 63452797, 77351967, 106890...",plasma membrane protein complex
GO:1904949,-0.570,-2.891,0.000e+00,0.000,117,47,"[63078357, 6924770, 63452797, 77351967, 106890...",ATPase complex
GO:0044459,-0.570,-2.890,0.000e+00,0.000,119,47,"[63078357, 6924770, 63452797, 77351967, 106890...",plasma membrane part
GO:0098796,-0.366,-1.959,5.537e-04,0.004,171,56,"[1994037, 32363526, 1809293, 268176, 13568664,...",membrane protein complex
GO:0043234,-0.366,-1.952,1.405e-03,0.003,180,56,"[1994037, 32363526, 1809293, 268176, 13568664,...",protein complex
GO:0016020,-0.551,-1.887,6.812e-03,0.005,75,13,"[21357752, 21687303, 63780068, 166593522, 1671...",membrane
GO:0044425,-0.332,-1.818,1.959e-03,0.008,191,62,"[1994037, 33721585, 21687303, 32363526, 180929...",membrane part
